# TDT4173 - Machine learning project, fall 2021 (short notebook)

In [1]:
# import libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from scipy.stats.stats import pearsonr

from flaml import AutoML
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

import moscow_housing_utils as mhu

SEED = 123 # for reproducibility
np.random.seed(SEED)
sns.set_style('darkgrid')
pd.set_option('display.max_colwidth', None) # don't truncate cell width
pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
# load and merge data
apartments = pd.read_csv('../data/apartments_train.csv')
buildings = pd.read_csv('../data/buildings_train.csv')
data = pd.merge(apartments, buildings.set_index('id'), how='left', left_on='building_id', right_index=True)

apartments_test = pd.read_csv('../data/apartments_test.csv')
buildings_test = pd.read_csv('../data/buildings_test.csv')
data_test = pd.merge(apartments_test, buildings_test.set_index('id'), how='left', left_on='building_id', right_index=True)

In [3]:
def data_wrangling_optimal(all_data):
    """
    Perform the best data wrangling we know (for LGBM) on all data we have
    """
    X = all_data.copy()
    
    # impute seller (negative value is interpreted as missing by LightGBM)
    X['seller'] = X['seller'].fillna(-1.0)
    
    X = mhu.delete_suspicious_data(X)

    # impute areas
    quants = X.area_total.quantile([0.25, 0.5, 0.75])
    (q1, q2, q3) = (quants.iloc[0], quants.iloc[1], quants.iloc[2])

    X.loc[X.area_kitchen.isnull() & (X.area_total <= q1), 'area_kitchen'] = \
        X.loc[X.area_total <= q1, 'area_kitchen'].median()
    X.loc[X.area_kitchen.isnull() & (q1 < X.area_total) & (X.area_total
          <= q2), 'area_kitchen'] = X.loc[(q1 < X.area_total)
            & (X.area_total <= q2), 'area_kitchen'].median()
    X.loc[X.area_kitchen.isnull() & (q2 < X.area_total) & (X.area_total
          <= q3), 'area_kitchen'] = X.loc[(q2 < X.area_total)
            & (X.area_total <= q3), 'area_kitchen'].median()
    X.loc[X.area_kitchen.isnull() & (q3 < X.area_total), 'area_kitchen'] = \
        X.loc[q3 < X.area_total, 'area_kitchen'].median()

    X.loc[X.area_living.isnull() & (X.area_total <= q1), 'area_living'] = \
        X.loc[X.area_total <= q1, 'area_living'].median()
    X.loc[X.area_living.isnull() & (q1 < X.area_total) & (X.area_total
          <= q2), 'area_living'] = X.loc[(q1 < X.area_total)
            & (X.area_total <= q2), 'area_living'].median()
    X.loc[X.area_living.isnull() & (q2 < X.area_total) & (X.area_total
          <= q3), 'area_living'] = X.loc[(q2 < X.area_total)
            & (X.area_total <= q3), 'area_living'].median()
    X.loc[X.area_living.isnull() & (q3 < X.area_total), 'area_living'] = \
        X.loc[q3 < X.area_total, 'area_living'].median()

    # fix imputed values that make no sense
    X.loc[X.area_total < X.area_kitchen, 'area_kitchen'] = X.area_total \
        * (X.area_kitchen.median() / X.area_total.median())
    X.loc[X.area_total < X.area_living, 'area_living'] = X.area_total \
        * (X.area_living.median() / X.area_total.median())

    # encode street
    X.street, _ = X.street.factorize()

    # impute ceiling
    X['ceiling'] = X['ceiling'].fillna(X['ceiling'].median())

    # impute layout
    X['layout'] = X['layout'].fillna(-1.0)

    # impute bathrooms with most reasonable combination to assume
    X['bathrooms_private'] = X['bathrooms_private'].fillna(1.0)
    X['bathrooms_shared'] = X['bathrooms_shared'].fillna(0.0)

    # impute windows with mode
    # (and also one of two most logical combinations)
    X['windows_court'] = X['windows_court'].fillna(1.0)
    X['windows_street'] = X['windows_street'].fillna(0.0)

    # impute balconies and loggias with modes
    X['balconies'] = X['balconies'].fillna(0.0)
    X['loggias'] = X['loggias'].fillna(1.0)

    # impute condition
    X['condition'] = X['condition'].fillna(-1)

    # impute phones with mode
    X['phones'] = X['phones'].fillna(1.0)

    # impute new-ness
    X['new'] = X['new'].fillna(0.0)

    # impute latitude and longitude with coordinates from Google maps
    X['latitude'] = X['latitude'].fillna(55.576675)
    X['longitude'] = X['longitude'].fillna(37.4868009)

    # impute district
    X.loc[X['district'].isna() & (X['latitude'] == 55.595160)
          & (X['longitude'] == 37.741109), 'district'] = 5.0
    X.loc[X['district'].isna() & (X['latitude'] == 55.576675)
          & (X['longitude'] == 37.486801), 'district'] = 11.0
    X.loc[X['district'].isna() & (X['latitude'] == 55.921627)
          & (X['longitude'] == 37.781578), 'district'] = 2.0
    X.loc[X['district'].isna() & (X['latitude'] == 55.583551)
          & (X['longitude'] == 37.711356), 'district'] = 5.0
    X.loc[X['district'].isna() & (X['latitude'] == 55.932127)
          & (X['longitude'] == 37.793705), 'district'] = 2.0
    # new category, to denote apartments outside Moscow
    X.loc[X['district'].isna(), 'district'] = 12.0

    # adjust year built (no effect for the tree-based algorithms)
    X['constructed'] = X['constructed'] - X['constructed'].min()
    X['constructed'] = X['constructed'].fillna(X['constructed'].median())

    # impute material with mode
    X['material'] = X['material'].fillna(2.0)

    # impute elevator data with mode of all three
    X['elevator_without'] = X['elevator_without'].fillna(0.0)
    X['elevator_passenger'] = X['elevator_passenger'].fillna(1.0)
    X['elevator_service'] = X['elevator_service'].fillna(0.0)
    X['elevator_score'] = -1 * X['elevator_without'] + X['elevator_service'
            ] + X['elevator_passenger']

    # impute parking (mode/median is 1.0)
    X['parking'] = X['parking'].fillna(1.0)

    # impute garbage chute system (mode is 1)
    X['garbage_chute'] = X['garbage_chute'].fillna(1.0)

    # impute heating
    X['heating'] = X['heating'].fillna(0.0)

    # engineer center_distance
    X['center_dist'] = list(zip(X.latitude, X.longitude))
    X['center_dist'] = X['center_dist'].apply(mhu.dist)
    
    # engineer distance_loc_[1, 2, 3, 4]
    X['distance_loc_1'] = list(zip(X.latitude, X.longitude))
    X['distance_loc_1'] = X['distance_loc_1'].apply(mhu.dist_loc_1)
    X['distance_loc_2'] = list(zip(X.latitude, X.longitude))
    X['distance_loc_2'] = X['distance_loc_2'].apply(mhu.dist_loc_2)
    X['distance_loc_3'] = list(zip(X.latitude, X.longitude))
    X['distance_loc_3'] = X['distance_loc_3'].apply(mhu.dist_loc_3)
    X['distance_loc_4'] = list(zip(X.latitude, X.longitude))
    X['distance_loc_4'] = X['distance_loc_4'].apply(mhu.dist_loc_4)
    
    # engineer direction
    X['direction'] = list(zip(X.latitude, X.longitude))
    X['direction'] = X['direction'].apply(mhu.direction)
    
    # Engineer district_center_dist
    X['district_center_dist'] = list(zip(X.latitude, X.longitude, X.district))
    X['district_center_dist'] = X['district_center_dist'].apply(mhu.dist_dist)

    return X

def update_train_test(X_train, X_test, all_data, drop_features):
    """
    Replace values in X_train and X_test with corresponding values
    in all_data
    """
    # add new features train and test datasets
    new_features = list(set(all_data.columns) - set(X_train.columns))
    X_train[new_features] = np.NaN
    X_test[new_features] = np.NaN
    
    # drop any original features
    all_data.drop(labels=drop_features, axis=1, inplace=True)
    X_train.drop(labels=drop_features, axis=1, inplace=True)
    X_test.drop(labels=drop_features, axis=1, inplace=True)

    X_train.set_index('id', inplace=True)
    X_test.set_index('id', inplace=True)
    X_train.update(all_data.set_index('id'))
    X_test.update(all_data.set_index('id'))
    X_train.reset_index()
    X_test.reset_index()

def plot_feature_importances(model, model_name, cols):
    """
    Analyse feature importance using mean decrease in impurity (MDI)
    """
    importances = pd.Series(model.feature_importances_, index=cols)
    fig, ax = plt.subplots(figsize=(18, 15))
    importances.plot.bar(ax=ax)
    ax.set_title(f"{model_name} - MDI / Gini Importance")
    ax.set_ylabel("Mean decrease in impurity")
    fig.tight_layout()

def plot_map(data, column='price', title='', ax=None, s=5, a=0.75, q_lo=0.0, q_hi=0.9, cmap='autumn'):
    """
    Plot 'column' in 'data' on Moscow map backdrop
    """
    data = data[['latitude', 'longitude', column]].sort_values(by=column, ascending=True)
    if not title:
        title = f"{column.title()} by location"
    backdrop = plt.imread('../data/moscow.png')
    backdrop = np.einsum('hwc, c -> hw', backdrop, [0, 1, 0, 0]) ** 2
    if ax is None:
        plt.figure(figsize=(12, 8), dpi=100)
        ax = plt.gca()
    discrete = data[column].nunique() <= 20
    if not discrete:
        lo, hi = data[column].quantile([q_lo, q_hi])
        hue_norm = plt.Normalize(lo, hi)
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(lo, hi))
        sm.set_array([])
    else:
        hue_norm = None 
    ax.imshow(backdrop, alpha=0.5, extent=[37, 38, 55.5, 56], aspect='auto', cmap='bone', norm=plt.Normalize(0.0, 2))
    sns.scatterplot(x='longitude', y='latitude', hue=data[column].tolist(), ax=ax, s=s, alpha=a, palette=cmap, linewidth=0, hue_norm=hue_norm, data=data)
    ax.set_xlim(37, 38)    # min/max longitude of image 
    ax.set_ylim(55.5, 56)  # min/max latitude of image
    if not discrete:
        ax.legend().remove()
        ax.figure.colorbar(sm)
    ax.set_title(title)
    return ax, hue_norm

def plot_predictions_on_map(X_test, y_test):
    """
    Plot predicted prices on Moscow map backdrop
    """
    data = X_test.copy()
    data["price"] = y_test
    plot_map(data, title="Predicted price by location")

In [4]:
# create list for storing all models' predictions
MODELS_NUM = 5
preds = [None] * MODELS_NUM

int_cols = [
    'seller', 'floor', 'rooms', 'layout', 'bathrooms_shared',
    'bathrooms_private', 'windows_court', 'windows_street', 'balconies',
    'loggias', 'condition', 'phones', 'new', 'district', 'constructed',
    'material', 'stories', 'parking', 'garbage_chute', 'heating',
    'elevator_score', 'street'
]
category_cols = ['seller', 'layout', 'district', 'heating', 'material',
                 'street', 'condition']

## Model 0

In [5]:
# create train and test datasets
unused_features = ['address', 'building_id']
X_train = data.drop(unused_features + ['price'], axis=1)
y_train = np.log1p(data.price)
X_test = data_test.drop(unused_features, axis=1)

# merge train and test for combined data processing
X = X_train.append(X_test, ignore_index=True)
X = data_wrangling_optimal(X)

# update train and test sets
drop_features = ['elevator_without', 'elevator_service', 'elevator_passenger']
update_train_test(X_train, X_test, all_data=X, drop_features=drop_features)

# convert columns to proper dtype
X_train[int_cols] = X_train[int_cols].astype('int32')
X_test[int_cols] = X_test[int_cols].astype('int32')

for col in category_cols:
    X_train[col] = pd.Categorical(X_train[col])
    X_test[col] = pd.Categorical(X_test[col])
cat_idx = [X_train.columns.get_loc(col) for col in category_cols]

X_train_model0 = X_train.copy()
X_test_model0 = X_test.copy()

In [6]:
model0 = LGBMRegressor(
    colsample_bytree=0.8346198485840488,
    learning_rate=0.038772388344039496,
    max_bin=1023,
    min_child_samples=17,
    n_estimators=23284,
    num_leaves=38,
    reg_alpha=0.003539069387982963,
    reg_lambda=7.774356662353698,
    verbose=-1,
    categorical_feature=cat_idx,
    random_state=SEED
)
model0.fit(X_train_model0, y_train)
preds[0] = np.floor(np.expm1(model0.predict(X_test_model0)))

C:\Users\matey\miniconda3\envs\mlcourse\lib\site-packages\lightgbm\basic.py:1222: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning('{0} keyword has been found in `params` and will be ignored.\n'
C:\Users\matey\miniconda3\envs\mlcourse\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_feature in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


## Model 1

In [7]:
# create train and test datasets
unused_features = ['address', 'building_id']
X_train = data.drop(unused_features + ['price'], axis=1)
y_train = np.log1p(data.price)
X_test = data_test.drop(unused_features, axis=1)

# merge train and test for combined data processing
X = X_train.append(X_test, ignore_index=True)
X = data_wrangling_optimal(X)

##############################################################################
# change what is different in data processing from best LGBM pipeline

# increase the value of underground parking
X.loc[X['parking'] == 1.0, 'parking'] = 4.0
# boolean heating - central or not
X['heating'] = (X['heating'] == 0.0)
# engineer percieved area
X['percieved_area'] = 2 * X['area_living'] + X['area_kitchen'] \
    + 0.5 * (X['area_total'] - X['area_living'] - X['area_kitchen'])
# engineer brightness
X['brightness'] = 2*X['balconies'] + X['loggias']
# engineer ratio living/total area
X['spaciousness'] = ( X.area_living / X.rooms).round(decimals=3)

# engineer price per square meter per district
X_train['sq_meter_price'] = data['price']/X_train['area_total']
X_train['sq_meter_price'] = X_train.groupby('district')['sq_meter_price'] \
    .transform(lambda x: round(x.median(), 2))
dist_medians = X_train[['district', 'sq_meter_price']].drop_duplicates()
d = dist_medians.set_index('district').T.to_dict('records').pop()
X_test['sq_meter_price'] = np.nan
X_test['sq_meter_price'] = X_test['district'].apply(lambda x: d.get(x))
# imputing the median for the cheapest district
# for apartments outside Moscow
X_test['sq_meter_price'] = X_test['sq_meter_price'].fillna(d.get(10.0))
# engineer price based on square meter price (target encoding)
X_train['price_enc'] = X_train.sq_meter_price * X_train.area_total
X_test['price_enc'] = X_test.sq_meter_price * X_test.area_total

# update train and test sets
drop_features = ['elevator_without', 'elevator_service', 'elevator_passenger',
                 'area_living', 'area_kitchen', 'district', 'rooms',
                 'latitude', 'longitude', 'area_total', 'ceiling']
update_train_test(X_train, X_test, all_data=X, drop_features=drop_features)
##############################################################################

# convert columns to proper dtype
ints = int_cols.copy()
ints.remove('district')
ints.remove('rooms')
X_train[ints] = X_train[ints].astype('int32')
X_test[ints] = X_test[ints].astype('int32')

cats = category_cols.copy()
cats.remove('district')
for col in cats:
    X_train[col] = pd.Categorical(X_train[col])
    X_test[col] = pd.Categorical(X_test[col])
cat_idx = [X_train.columns.get_loc(col) for col in cats]

X_train_model1 = X_train.copy()
X_test_model1 = X_test.copy()

In [8]:
model1 = LGBMRegressor(
    colsample_bytree=0.8346198485840488,
    learning_rate=0.038772388344039496,
    max_bin=1023,
    min_child_samples=17,
    n_estimators=23284,
    num_leaves=38,
    reg_alpha=0.003539069387982963,
    reg_lambda=7.774356662353698,
    verbose=-1,
    categorical_feature=cat_idx,
    random_state=SEED
)
model1.fit(X_train_model1, y_train)
preds[1] = np.floor(np.expm1(model1.predict(X_test_model1)))

In [21]:
# check correlation between predictions
pearsonr(preds[0], preds[1]) # we mainly care about first number

(0.976561085472111, 0.0)

## Model 2

In [9]:
# create train and test datasets
unused_features = ['address', 'building_id']
X_train = data.drop(unused_features + ['price'], axis=1)
y_train = np.log1p(data.price)
X_test = data_test.drop(unused_features, axis=1)

# merge train and test for combined data processing
X = X_train.append(X_test, ignore_index=True)
X = data_wrangling_optimal(X)

##############################################################################
# change what is different in data processing from best LGBM pipeline

# OHE categorical features
ints = int_cols.copy()
ints.remove('street')
cats = category_cols.copy()
cats.remove('street')
X = pd.get_dummies(X, columns=cats)

# update train and test sets
drop_features = ['elevator_without', 'elevator_service', 'elevator_passenger',
                 'street']
update_train_test(X_train, X_test, all_data=X, drop_features=drop_features)
X_train.drop(cats, axis=1, inplace=True)
X_test.drop(cats, axis=1, inplace=True)
##############################################################################

X_train_model2 = X_train.copy()
X_test_model2 = X_test.copy()

In [10]:
model2 = RandomForestRegressor(
    max_features=0.6913722322623973,
    max_leaf_nodes=23284,
    n_estimators=667,
    n_jobs=-1,
    random_state=SEED
)
model2.fit(X_train_model2, y_train)
preds[2] = np.floor(np.expm1(model2.predict(X_test_model2)))

In [22]:
# check correlation between predictions
pearsonr(preds[0], preds[2])

(0.9604991464923318, 0.0)

# Model 3

In [11]:
# create train and test datasets
unused_features = ['address', 'building_id']
X_train = data.drop(unused_features + ['price'], axis=1)
y_train = np.log1p(data.price)
X_test = data_test.drop(unused_features, axis=1)

# merge train and test for combined data processing
X = X_train.append(X_test, ignore_index=True)
X = data_wrangling_optimal(X)

# update train and test sets
drop_features = ['elevator_without', 'elevator_service', 'elevator_passenger']
update_train_test(X_train, X_test, all_data=X, drop_features=drop_features)

# convert columns to proper dtype
X_train[int_cols] = X_train[int_cols].astype('int32')
X_test[int_cols] = X_test[int_cols].astype('int32')

for col in category_cols:
    X_train[col] = pd.Categorical(X_train[col])
    X_test[col] = pd.Categorical(X_test[col])
cat_idx = [X_train.columns.get_loc(col) for col in category_cols]

X_train_model3 = X_train.copy()
X_test_model3 = X_test.copy()

In [12]:
model3 = CatBoostRegressor(
    n_estimators=1889,
    learning_rate=0.2,
    thread_count=-1,
    depth=6,
    silent=True,
    bagging_temperature=0.2,
    early_stopping_rounds=63,
    cat_features=cat_idx,
    allow_writing_files=False,
    random_seed=SEED
)
model3.fit(X_train_model3, y_train)
preds[3] = np.floor(np.expm1(model3.predict(X_test_model3)))

In [23]:
# check correlation between predictions
pearsonr(preds[0], preds[3])

(0.9516869986471448, 0.0)

## Model 4

In [13]:
model4 = GradientBoostingRegressor(
    n_estimators=3000,
    learning_rate=0.05,
    max_depth=4,
    max_features='sqrt',
    min_samples_leaf=15,
    min_samples_split=10, 
    loss='huber',
    random_state=SEED
)

model4.fit(X_train_model0, y_train)
preds[4] = np.floor(np.expm1(model4.predict(X_test_model0)))

In [24]:
pearsonr(preds[0], preds[4])

(0.9764273067341966, 0.0)

## Stacking

In [14]:
# create dataframe for easier calculation and storing of results
preds_df = pd.DataFrame(preds)
preds_df = preds_df.transpose()
preds_df.columns = ['lgbm_best', 'lgbm_mod', 'random_forest', 'catboost', 'gradient_boost']

In [15]:
# simple mean
preds_df["average"] = preds_df.mean(axis = 1)

The simple mean of the 5 used models is corresponding to our best result on Kaggle.

In [16]:
ntrain = X_train.shape[0]
ntest = X_test.shape[0]
NFOLDS = 5 # set number of folds for out-of-fold prediction
skf = StratifiedKFold(
    n_splits=NFOLDS,
    shuffle=True,
    random_state=SEED
) # K-Folds cross-validator

def get_oof(clf, x_train, y_train, x_test, outlier_limit=20):
    """
    Popular function on Kaggle.
    
    Trains a classifier on 4/5 of the training data and
    predicts the rest (1/5). This procedure is repeated for all 5 folds,
    thus we have predictions for all training set. This prediction is one
    column of meta-data, later on used as a feature column by a meta-algorithm.
    We predict the test part and average predictions across all 5 models.
    
    Keyword arguments:
    clf -- classifier
    x_train -- 4/5 of training data
    y_train -- corresponding labels
    x_test -- all test data
    outlier_limit -- outlier limit for rounded price log
    
    """
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    y_strat = y_train.round()
    # bundle all high outliers in one class
    y_strat[y_strat > outlier_limit] = outlier_limit + 1

    for i, (train_index, test_index) in enumerate(skf.split(x_train, y_strat)):
        x_tr = x_train.iloc[train_index]
        y_tr = y_train.iloc[train_index]
        x_te = x_train.iloc[test_index]

        clf.fit(x_tr, y_tr)
        
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [17]:
m0_oof_train, m0_oof_test = get_oof(model0, X_train_model0, y_train, X_test_model0)
m1_oof_train, m1_oof_test = get_oof(model1, X_train_model1, y_train, X_test_model1)
m2_oof_train, m2_oof_test = get_oof(model2, X_train_model2, y_train, X_test_model2)
m3_oof_train, m3_oof_test = get_oof(model3, X_train_model3, y_train, X_test_model3)
m4_oof_train, m4_oof_test = get_oof(model4, X_train_model0, y_train, X_test_model0)

# First-level output as new features

x_train = np.concatenate((
    m0_oof_train,
    m1_oof_train,
    m2_oof_train,
    m3_oof_train,
    m4_oof_train
), axis=1)

x_test = np.concatenate((
    m0_oof_test,
    m1_oof_test,
    m2_oof_test,
    m3_oof_test,
    m4_oof_test
), axis=1)

meta_features = ['lgbm_best', 'lgbm_mod', 'random_forest', 'catboost', 'gradient_boost']
x_train_meta = pd.DataFrame(data=x_train, columns=meta_features)
x_test_meta = pd.DataFrame(data=x_test, columns=meta_features)

META_MODEL = ExtraTreesRegressor(max_features=0.6679422374183948, max_leaf_nodes=161,
                    n_estimators=154, n_jobs=-1, random_state=SEED)

META_MODEL.fit(x_train_meta, y_train)
preds_df["true_stacking"] = np.floor(np.expm1(META_MODEL.predict(x_test_meta)))

C:\Users\matey\miniconda3\envs\mlcourse\lib\site-packages\lightgbm\basic.py:1222: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning('{0} keyword has been found in `params` and will be ignored.\n'
C:\Users\matey\miniconda3\envs\mlcourse\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_feature in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
C:\Users\matey\miniconda3\envs\mlcourse\lib\site-packages\lightgbm\basic.py:1222: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning('{0} keyword has been found in `params` and will be ignored.\n'
C:\Users\matey\miniconda3\envs\mlcourse\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical

# Storing and retrieving predictions

In [18]:
# store dataframe with predictions
preds_df.to_csv('predictions.csv', header=True, index=False)

In [27]:
# retrieve predictions from csv
preds_df = pd.read_csv('predictions.csv', header=0)

# Final predictions

This is our best scoring submission dataframe. For reproducibility issues, please see note at the end of the long notebook.

In [19]:
# Construct submission dataframe for simple mean predictions
submission = pd.DataFrame()
submission['id'] = data_test.id
submission['price_prediction'] = preds_df["average"].values
print(f'Generated {len(submission)} predictions')

# Export submission to csv with headers
submission.to_csv('solution_1.csv', index=False)

Generated 9937 predictions


This is the dataframe for our second chosen solution.

In [20]:
# Construct submission dataframe for ensemble predictions
submission = pd.DataFrame()
submission['id'] = data_test.id
submission['price_prediction'] = preds_df["true_stacking"].values
print(f'Generated {len(submission)} predictions')

# Export submission to csv with headers
submission.to_csv('solution_2.csv', index=False)

Generated 9937 predictions
